# Section 0 — Environment Setup

## 0.1 — Install Required Libraries

Description:
We’ll install the core dependencies for LangChain, OpenAI, Anthropic, and vector DBs (Chroma).

In [ ]:
# ======================================
# Section 0.1 — Install Required Libraries
# ======================================

!pip install -U langchain langchain-openai langchain-anthropic langchain-community \
              chromadb pypdf tiktoken langsmith python-dotenv


## 0.2 — Import Dependencies

Description:
We import the necessary libraries for this lecture, including LangChain, OpenAI, and Anthropic integrations.

In [2]:
# ======================================
# Section 0.2 — Import Dependencies
# ======================================

import os
from google.colab import userdata

# LangChain Core
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

## 0.3 — Configure API Keys

Description:
We’ll load API keys stored in Google Colab environment variables.
Before running, set your keys in:
Colab → Settings → Variables → Add OPENAI_API_KEY and ANTHROPIC_API_KEY.

In [3]:
# ======================================
# Section 0.3 — Configure API Keys
# ======================================

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")

# Optional: LangSmith tracing (debugging & observability)
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "langchain-lecture"

print("✅ API keys configured successfully!")


✅ API keys configured successfully!


## 0.4 — Verify Installation & LLM Connectivity

Description:
We’ll run a quick test to ensure both OpenAI and Anthropic models are working.

In [4]:
# ======================================
# Section 0.4 — Verify Installation & LLM Connectivity
# ======================================

# Test OpenAI connection
openai_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
response_openai = openai_llm.invoke("Say 'ready' if you can hear me.")
print("🔹 OpenAI LLM:", response_openai.content)

# Test Anthropic connection (optional)
try:
    anthropic_llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
    response_anthropic = anthropic_llm.invoke("Say 'ready' if you can hear me.")
    print("🔹 Anthropic LLM:", response_anthropic.content)
except Exception as e:
    print("⚠️ Anthropic test skipped (no API key or model unavailable).")


🔹 OpenAI LLM: Ready!
🔹 Anthropic LLM: Ready!


# Section 1 — Understanding LLMs (Large Language Models)
Goal: Understand what LLMs are, how LangChain integrates them, and how to initialize and test models from OpenAI and Anthropic.

## 1.1 — Initialize & Test OpenAI LLM

Description:
In this cell, we’ll set up and test an OpenAI chat model using LangChain’s ChatOpenAI wrapper.
We’ll explain key parameters, even if we don’t modify them yet.

In [5]:
# ======================================
# Section 1.1 — Initialize & Test OpenAI LLM
# ======================================

# Import the OpenAI chat model integration for LangChain
from langchain_openai import ChatOpenAI

# Create an OpenAI LLM instance
# --------------------------------------
# Parameters:
# - model: The specific OpenAI model to use.
#          Here we use "gpt-4o-mini" (fast, cost-effective, supports RAG).
# - temperature: Controls randomness of output.
#       0.0 → deterministic responses
#       1.0 → more creative and diverse responses
# - max_tokens: Maximum number of tokens the model can return in the response.
# - timeout: Optional timeout for API call (not used here).
# - api_key: Uses the API key we configured earlier via environment variables.
openai_llm = ChatOpenAI(
    model="gpt-4o-mini",   # Recommended for fast, cost-effective RAG tasks
    temperature=0.2,       # Low randomness for more precise answers
    max_tokens=500         # Reasonable cap for lecture demos
)

# Quick test to verify OpenAI is working
response = openai_llm.invoke("Explain in one sentence what LangChain is.")
print("🔹 OpenAI LLM Response:\n", response.content)


🔹 OpenAI LLM Response:
 LangChain is a framework designed to facilitate the development of applications that leverage large language models by providing tools for chaining together various components like prompts, memory, and APIs.


## Cell 1.2 — Initialize & Test Anthropic LLM
Description:
In this cell, we’ll set up and test an Anthropic chat model using LangChain’s ChatAnthropic wrapper.
Anthropic’s models (e.g., Claude 3) are great for reasoning, summarization, and multi-step problem solving.

In [6]:
# ======================================
# Section 1.2 — Initialize & Test Anthropic LLM
# ======================================

# Import the Anthropic chat model integration for LangChain
from langchain_anthropic import ChatAnthropic

# Create an Anthropic LLM instance
# --------------------------------------
# Parameters:
# - model: The Anthropic Claude model to use.
#          Here we use "claude-3-haiku-20240307" (fast, cost-effective).
#          Alternative: "claude-3-sonnet-20240229" (higher quality, slower).
# - temperature: Controls creativity.
#       0.0 → deterministic responses
#       1.0 → more creative and diverse outputs
# - max_tokens: Maximum number of tokens in the response.
# - api_key: Uses the API key configured earlier via environment variables.
anthropic_llm = ChatAnthropic(
    model="claude-3-haiku-20240307",  # Optimized for speed and low latency
    temperature=0.2,                  # Keep responses focused and consistent
    max_tokens=500                    # Reasonable cap for demos
)

# Quick test to verify Anthropic is working
try:
    response = anthropic_llm.invoke("Explain in one sentence what LangChain is.")
    print("🔹 Anthropic LLM Response:\n", response.content)
except Exception as e:
    print("⚠️ Anthropic model test skipped:", e)


🔹 Anthropic LLM Response:
 LangChain is a framework for building applications with large language models (LLMs) that provides a set of abstractions and tools to make it easier to build applications that interact with LLMs.


# Section 2 — Prompt Engineering with LangChain
Goal: Learn how to create effective prompts using LangChain’s PromptTemplate and ChatPromptTemplate to control LLM behavior.

## 2.1 — Using PromptTemplate (Single-Turn Prompts)

Description:
In this cell, we’ll create a basic PromptTemplate in LangChain, which lets us define a prompt structure with placeholders that are later filled with dynamic inputs.
PromptTemplate is ideal for single-turn interactions where you send one prompt and get one response.

In [7]:
# ======================================
# Section 2.1 — Using PromptTemplate (Single-Turn Prompts)
# ======================================

# Import PromptTemplate from LangChain
from langchain.prompts import PromptTemplate

# Create a simple prompt template
# ------------------------------------------------------
# Parameters:
# - input_variables: A list of placeholder names that will be dynamically replaced.
# - template: The actual prompt text with placeholders in curly braces {}.
# ------------------------------------------------------
simple_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms for a beginner."
)

# Test the prompt template by formatting it with an example input
formatted_prompt = simple_prompt.format(topic="LangChain")
print("🔹 Formatted Prompt:\n", formatted_prompt)

# Send the formatted prompt to OpenAI LLM for testing
response = openai_llm.invoke(formatted_prompt)
print("\n🔹 LLM Response:\n", response.content)


🔹 Formatted Prompt:
 Explain LangChain in simple terms for a beginner.

🔹 LLM Response:
 LangChain is a framework designed to help developers build applications that use language models, like those from OpenAI, in a more structured and efficient way. Here’s a simple breakdown:

1. **Language Models**: These are AI systems that can understand and generate human-like text. They can answer questions, write stories, summarize information, and more.

2. **Building Blocks**: LangChain provides various components (or "chains") that you can use to create applications. These components can handle tasks like retrieving information, processing text, and managing conversations.

3. **Integration**: It helps you connect different parts of your application easily. For example, you can combine a language model with a database to fetch information and then use the model to generate a response based on that data.

4. **Flexibility**: LangChain is designed to be flexible, allowing developers to customiz

## 2.2 — Using ChatPromptTemplate (Multi-Turn Conversational Prompts)
Description:
Unlike PromptTemplate, which creates a single string prompt,
ChatPromptTemplate is designed for chat-based models like OpenAI’s GPT and Anthropic’s Claude.
It allows you to structure prompts using roles (system, human, ai) and makes multi-turn conversations easier to manage.

In [8]:
# ======================================
# Section 2.2 — Using ChatPromptTemplate (Multi-Turn Conversational Prompts)
# ======================================

# Import ChatPromptTemplate from LangChain
from langchain.prompts import ChatPromptTemplate

# Create a chat-based prompt template
# ------------------------------------------------------
# Here we define two message roles:
# 1. System message → sets the assistant's behavior/personality.
# 2. Human message → takes a dynamic input from the user.
# ------------------------------------------------------
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert AI tutor who explains concepts simply and clearly."),
    ("human", "Explain {topic} in simple terms for a beginner.")
])

# Test the chat prompt template by formatting it with an example input
formatted_chat_prompt = chat_prompt.format_messages(topic="LangChain")

# The formatted prompt will be a list of structured messages with roles
print("🔹 Formatted Chat Messages:\n", formatted_chat_prompt)

# Send the chat prompt to OpenAI LLM for testing
response = openai_llm.invoke(formatted_chat_prompt)
print("\n🔹 LLM Response:\n", response.content)


🔹 Formatted Chat Messages:
 [SystemMessage(content='You are an expert AI tutor who explains concepts simply and clearly.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain LangChain in simple terms for a beginner.', additional_kwargs={}, response_metadata={})]

🔹 LLM Response:
 Sure! LangChain is a framework designed to help developers build applications that use language models, like those from OpenAI (such as ChatGPT). Think of it as a toolkit that makes it easier to create programs that can understand and generate human-like text.

Here are some key points to understand LangChain:

1. **Language Models**: At its core, LangChain works with language models, which are AI systems trained to understand and generate text. These models can answer questions, write stories, summarize information, and more.

2. **Chains**: The term "chain" in LangChain refers to the idea of linking together different tasks or steps in a process. For example, you might first ask a que

# Section 3 — Working with Chains in LangChain
🎯 Goal

Students will learn:

What chains are and why we need them.

How to build simple LLM chains.

How to combine prompt templates + LLMs.

How to extend chains for retrieval (RAG) later.

## 3.1 — Introduction to Chains & First Simple LLMChain

Description:
Chains in LangChain allow us to connect multiple components (like prompt templates, LLMs, retrievers, memory, etc.) into a single pipeline.
Without chains, we’d have to:

Manually format the prompt

Call the LLM directly

Parse the result ourselves

With chains, LangChain automates these steps and makes the workflow cleaner and reusable.

In this demo, we’ll create a basic LLMChain that connects:
PromptTemplate → OpenAI LLM → Final Answer

In [9]:
# ======================================
# Section 3.1 — Introduction to Chains (New Runnable Interface)
# ======================================

from langchain.prompts import PromptTemplate

# Step 1: Define a prompt template
# ------------------------------------------------------
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in one sentence, as if teaching a beginner."
)

# Step 2: Combine Prompt + LLM into a runnable chain
# ------------------------------------------------------
# Instead of using LLMChain, we now connect components using the | operator.
chain = prompt | openai_llm

# Step 3: Run the chain with an example input
topic = "LangChain"
response = chain.invoke({"topic": topic})

print(f"🔹 Prompted Topic: {topic}")
print("🔹 LLM Response:\n", response.content)


🔹 Prompted Topic: LangChain
🔹 LLM Response:
 LangChain is a framework that helps developers build applications using language models by providing tools to manage prompts, handle data, and integrate with various APIs and services.


## 3.2 — Multi-Input Chain Using PromptTemplate | LLM

Description:
In this cell, we’ll create a prompt that accepts two variables instead of one.
This helps students understand how to handle dynamic inputs for more complex prompt engineering.

In [10]:
# ======================================
# Section 3.2 — Multi-Input Chain Using PromptTemplate | LLM
# ======================================

from langchain.prompts import PromptTemplate

# Step 1: Create a multi-variable prompt template
# ------------------------------------------------------
# Parameters:
# - input_variables: List of placeholders we want to replace dynamically.
# - template: The prompt with two placeholders: {concept} and {audience}.
prompt = PromptTemplate(
    input_variables=["concept", "audience"],
    template="Explain {concept} to a {audience} in two sentences."
)

# Step 2: Combine prompt + LLM into a runnable chain
# ------------------------------------------------------
chain = prompt | openai_llm

# Step 3: Run the chain with dynamic inputs
inputs = {
    "concept": "LangChain",
    "audience": "high school student"
}

response = chain.invoke(inputs)

# Step 4: Show the results
print(f"🔹 Concept: {inputs['concept']}")
print(f"🔹 Audience: {inputs['audience']}")
print("\n🔹 LLM Response:\n", response.content)


🔹 Concept: LangChain
🔹 Audience: high school student

🔹 LLM Response:
 LangChain is a tool that helps developers create applications using language models, like chatbots or virtual assistants, by connecting different parts of the software together. It makes it easier to build complex systems that can understand and generate human-like text, allowing for more interactive and intelligent user experiences.


## 3.3 — Sequential Chains Using RunnableSequence

Description:
Sometimes, you need multiple reasoning steps.
For example:

Step 1: Generate an outline for a topic.

Step 2: Summarize that outline into a short description.

Step 3: Produce the final answer in a clean format.

Instead of calling the LLM separately for each step, we chain them together into one pipeline.

In [15]:
# ======================================
# Section 3.3 — Sequential Chains Using Runnable Composition
# ======================================

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# Step 1: Prompt for generating an outline
outline_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Create a short outline with 3 bullet points about {topic}."
)

# Step 2: Prompt for summarizing the outline
summary_prompt = PromptTemplate(
    input_variables=["outline"],
    template="Summarize the following outline into two sentences:\n\n{outline}"
)

# Step 3: Create the chain using dictionary mapping
# ------------------------------------------------------
# Explanation:
# 1. First, we generate the outline → outline_prompt | openai_llm
# 2. Then, we pass that outline to summary_prompt | openai_llm
# 3. RunnablePassthrough lets us carry variables forward without recomputing them.
chain = (
    {"outline": outline_prompt | openai_llm}  # Step 1: Generate outline
    | summary_prompt                         # Step 2: Inject outline into summary prompt
    | openai_llm                             # Step 3: Summarize
)

# Step 4: Run the chain
topic = "LangChain framework"
response = chain.invoke({"topic": topic})

# Step 5: Show the result
print(f"🔹 Topic: {topic}")
print("\n🔹 Final Summary:\n", response.content)


🔹 Topic: LangChain framework

🔹 Final Summary:
 The LangChain framework is designed to simplify the integration of language models into various applications, featuring core components like chains, agents, and memory that facilitate complex workflows. It has practical applications in areas such as chatbots, content generation, and data analysis, enhancing productivity and creativity in leveraging language models for diverse tasks.


## 3.4 — Advanced Multi-Step Reasoning Chain

Description:
This example demonstrates how to combine multiple prompts + LLMs into a single pipeline,
where each step depends on the output of the previous one.

In [11]:
# ======================================
# Section 3.4 — Advanced Multi-Step Reasoning Chain (FINAL FIX)
# ======================================

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# Step 1: Prompt for generating 3 creative ideas
ideas_prompt = PromptTemplate(
    input_variables=["topic"],
    template=(
        "Generate 3 creative and unique ideas related to the topic: {topic}.\n"
        "Number each idea clearly."
    )
)

# Step 2: Prompt for choosing the best idea
choose_best_prompt = PromptTemplate(
    input_variables=["ideas"],
    template=(
        "You are a critical evaluator. Review the following 3 ideas:\n\n{ideas}\n\n"
        "Select the SINGLE best idea and explain briefly why you chose it."
    )
)

# Step 3: Prompt for expanding the best idea into an article
expand_prompt = PromptTemplate(
    input_variables=["best_idea"],
    template="Write a short, engaging article (5-6 sentences) based on this idea:\n\n{best_idea}"
)

# ----------------------------
# Step 4: Build the chain properly
# ----------------------------
# First: Generate ideas from topic
generate_ideas_chain = ideas_prompt | openai_llm

# Second: Choose the best idea from generated ideas
choose_best_chain = (
    {"ideas": generate_ideas_chain}           # Take output from first step
    | choose_best_prompt
    | openai_llm
)

# Third: Expand the chosen best idea into an article
final_chain = (
    {"best_idea": choose_best_chain}          # Pass the best idea forward
    | expand_prompt
    | openai_llm
)

# ----------------------------
# Step 5: Run the chain
# ----------------------------
topic = "Applications of LangChain in real-world business"
response = final_chain.invoke({"topic": topic})

# ----------------------------
# Step 6: Display result
# ----------------------------
print(f"🔹 Topic: {topic}")
print("\n🔹 Final Article:\n")
print(response.content)


🔹 Topic: Applications of LangChain in real-world business

🔹 Final Article:

In today's fast-paced business landscape, **Dynamic Customer Support Chatbots** are revolutionizing the way companies interact with their customers. These advanced chatbots, powered by LangChain, offer personalized and context-aware responses that significantly enhance the customer experience. By integrating with CRM systems, they gain valuable insights into customer needs and preferences, allowing them to provide tailored solutions and escalate issues when necessary. This not only streamlines operations by alleviating the workload on human agents but also fosters more engaging and responsive interactions. As these chatbots learn from each interaction, they continuously adapt to evolving customer expectations, ensuring businesses remain competitive. Ultimately, the implementation of dynamic chatbots can lead to immediate and substantial improvements in customer service operations, driving satisfaction and rete

# Section 4 — Working with Documents

Goal:
Teach students how to load PDFs, split them into chunks, and prepare them for retrieval-augmented generation (RAG).
This is the foundation for the next section where we’ll connect everything into a full RAG pipeline.

## 4.1 — Load and Preview a PDF

Description:
In this cell, we’ll let students upload a PDF into Colab, load it using LangChain’s PyPDFLoader, and preview the first page.

In [12]:
# ======================================
# Section 4.1 — Load and Preview a PDF
# ======================================

from google.colab import files
from langchain_community.document_loaders import PyPDFLoader

# Step 1: Upload a PDF
# ------------------------------------------------------
# Students select a file from their computer.
uploaded = files.upload()

# Get the uploaded file path
pdf_path = list(uploaded.keys())[0]
print(f"✅ Uploaded PDF: {pdf_path}")

# Step 2: Load the PDF using LangChain's PyPDFLoader
# ------------------------------------------------------
# PyPDFLoader splits the PDF into LangChain Document objects.
# - Each Document has .page_content (text) and .metadata (page info).
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# Step 3: Preview the document
print(f"✅ Total pages loaded: {len(pages)}\n")
print("🔹 Preview of the first page:\n")
print(pages[0].page_content[:800])  # Show first 800 characters


Saving 2025-01-18-pdf-1-TechAI-Goolge-whitepaper_Prompt Engineering_v4-af36dcc7a49bb7269a58b1c9b89a8ae1.pdf to 2025-01-18-pdf-1-TechAI-Goolge-whitepaper_Prompt Engineering_v4-af36dcc7a49bb7269a58b1c9b89a8ae1.pdf
✅ Uploaded PDF: 2025-01-18-pdf-1-TechAI-Goolge-whitepaper_Prompt Engineering_v4-af36dcc7a49bb7269a58b1c9b89a8ae1.pdf
✅ Total pages loaded: 65

🔹 Preview of the first page:

Prompt  
Engineering
Author: Lee Boonstra


## 4.2 — Split Documents into Chunks

Description:
In this cell, we’ll use LangChain’s RecursiveCharacterTextSplitter to split the PDF into manageable chunks.
Why we need chunking:

Embeddings work best with shorter text.

Smaller chunks improve recall during retrieval.

Overlapping chunks preserve context continuity.

In [13]:
# ======================================
# Section 4.2 — Split Documents into Chunks
# ======================================

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 1: Create a text splitter
# ------------------------------------------------------
# Parameters:
# - chunk_size: Maximum characters per chunk (adjust based on model context).
# - chunk_overlap: Number of characters repeated between chunks to preserve context.
# - separators: Order of preferred split points, largest to smallest.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,       # ~1-2 paragraphs per chunk
    chunk_overlap=200,     # Preserve context across chunks
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Step 2: Split the PDF pages into chunks
chunks = text_splitter.split_documents(pages)

# Step 3: Preview the result
print(f"✅ Total chunks created: {len(chunks)}")
print("\n🔹 First chunk preview:\n")
print(chunks[0].page_content[:500])  # Show first 500 characters
print("\n🔹 Metadata example:", chunks[0].metadata)


✅ Total chunks created: 103

🔹 First chunk preview:

Prompt  
Engineering
Author: Lee Boonstra

🔹 Metadata example: {'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.0 (Macintosh)', 'creationdate': '2024-10-31T13:52:57-06:00', 'moddate': '2024-10-31T13:53:02-06:00', 'trapped': '/False', 'source': '2025-01-18-pdf-1-TechAI-Goolge-whitepaper_Prompt Engineering_v4-af36dcc7a49bb7269a58b1c9b89a8ae1.pdf', 'total_pages': 65, 'page': 0, 'page_label': '1'}


## 4.3 — Generate Embeddings for Chunks

Description:
In this cell, we’ll use OpenAI’s text-embedding-3-small model via LangChain’s OpenAIEmbeddings class.
We generate embeddings for all chunks so we can later store them in a vector database.

In [14]:
# ======================================
# Section 4.3 — Generate Embeddings for Chunks
# ======================================

from langchain_openai import OpenAIEmbeddings

# Step 1: Initialize the embeddings model
# ------------------------------------------------------
# Parameters:
# - model: The OpenAI embeddings model to use.
#   Options:
#     "text-embedding-3-small"  → fast & cost-effective (~1,536 dimensions)
#     "text-embedding-3-large"  → more accurate (~3,072 dimensions, higher cost)
# - api_key: Uses the environment variable configured earlier.
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# Step 2: Generate embeddings for a sample query (sanity check)
sample_vector = embedding_model.embed_query("What is LangChain?")
print(f"✅ Sample embedding vector length: {len(sample_vector)}")
print(f"🔹 First 10 dimensions: {sample_vector[:10]}")

# Step 3: We'll use this embedding model later when storing chunks in a vector database
print("\n✅ Embedding model initialized successfully!")


✅ Sample embedding vector length: 1536
🔹 First 10 dimensions: [0.003030850552022457, -0.0035995321813970804, -0.020205670967698097, 0.0034851604141294956, 0.0002958574041258544, -0.0026686734054237604, -0.000631030066870153, 0.019671935588121414, -0.03939470276236534, -0.01785469613969326]

✅ Embedding model initialized successfully!


## 4.4 — Store Chunks in Chroma Vector Database

Description:
In this cell, we’ll use ChromaDB (a fast in-memory vector database) to store embeddings and their associated chunks.
Once stored, we’ll be able to query the database using semantic similarity search.

In [16]:
# ======================================
# Section 4.4 — Store Chunks in Chroma Vector Database
# ======================================

from langchain_community.vectorstores import Chroma

# Step 1: Create and populate Chroma DB
# ------------------------------------------------------
# Parameters:
# - documents: The chunks we generated earlier.
# - embedding: The OpenAI embeddings model.
# - persist_directory: If set to a folder path, Chroma will store data persistently.
#   Here we keep it in-memory for simplicity.
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=None   # Set to "./chroma_db" if you want to persist locally
)

print("✅ Chroma vector database created successfully!")

# Step 2: Quick test: semantic similarity search
# ------------------------------------------------------
query = "Explain the purpose of LangChain"
results = vectorstore.similarity_search(query, k=2)

print(f"\n🔹 Query: {query}\n")
print("🔹 Top 2 retrieved chunks:\n")
for i, doc in enumerate(results, 1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content[:300], "...\n")


✅ Chroma vector database created successfully!

🔹 Query: Explain the purpose of LangChain

🔹 Top 3 retrieved chunks:

--- Chunk 1 ---
agent.run(prompt)
Snippet 1. Creating a ReAct Agent with LangChain and VertexAI
Code	Snippet	2	shows	the	result.	Notice	that	ReAct	makes	a	chain	of	five	searches.	In	fact,	
the	LLM	is	scraping	Google	search	results	to	figure	out	the	band	names.	Then,	it	lists	the	
results	as	observations	and	chains	 ...

--- Chunk 2 ---
agent.run(prompt)
Snippet 1. Creating a ReAct Agent with LangChain and VertexAI
Code	Snippet	2	shows	the	result.	Notice	that	ReAct	makes	a	chain	of	five	searches.	In	fact,	
the	LLM	is	scraping	Google	search	results	to	figure	out	the	band	names.	Then,	it	lists	the	
results	as	observations	and	chains	 ...

--- Chunk 3 ---
Prompt Engineering
September 2024
29
Chain of Thought (CoT)
Chain of Thought (CoT) 9	prompting	is	a	technique	for	improving	the	reasoning	capabilities	
of LLMs by generating intermediate reasoning steps.	This	helps	the	LLM

# Section 5 — Working with Memory

Goal
Teach students how to add memory to LangChain applications so LLMs can remember previous user inputs and maintain conversational context.

## 5.1 — Using LLMs Without Memory

Description:
By default, LLMs are stateless — they don’t remember previous messages unless you explicitly resend the context every time.

In [17]:
# ======================================
# Section 5.1 — Using LLMs Without Memory
# ======================================

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Step 1: Create a simple LLM instance
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Step 2: Create a simple chat prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

# Step 3: Build a simple runnable chain (no memory)
chain_no_memory = prompt | llm

# Step 4: Ask two related questions
q1 = "My name is Alex."
q2 = "What is my name?"

resp1 = chain_no_memory.invoke({"input": q1})
resp2 = chain_no_memory.invoke({"input": q2})

print("=== Without Memory ===\n")
print(f"Q1: {q1}\nA1: {resp1.content}\n")
print(f"Q2: {q2}\nA2: {resp2.content}\n")


=== Without Memory ===

Q1: My name is Alex.
A1: Nice to meet you, Alex! How can I assist you today?

Q2: What is my name?
A2: I'm sorry, but I don't have access to personal information about you unless you share it with me. How can I assist you today?



## 5.2 — Using ConversationBufferMemory

Description:
Now we’ll use ConversationBufferMemory so the LLM remembers the full conversation history.

In [18]:
# ======================================
# Section 5.2 — Using ConversationBufferMemory
# ======================================

from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Step 1: Create a prompt that EXPLICITLY shows chat history



# Updated prompt with explicit chat_history variable
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("ai", "This is the conversation so far:\n{chat_history}"),
    ("human", "{input}")
])


# Step 1: Create memory object
memory = ConversationBufferMemory(
    memory_key="chat_history",   # Inject history into {chat_history} variable
    return_messages=True
)

# Step 2: Create LLMChain with memory
chain_with_memory = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# Step 3: Ask two related questions
q1 = "My name is Alex."
q2 = "What is my name?"

resp1_mem = chain_with_memory.invoke({"input": q1})
resp2_mem = chain_with_memory.invoke({"input": q2})

print("=== With Memory (Updated Prompt) ===\n")
print(f"Q1: {q1}\nA1: {resp1_mem['text']}\n")
print(f"Q2: {q2}\nA2: {resp2_mem['text']}\n")


/tmp/ipython-input-2131633540.py:22: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/tmp/ipython-input-2131633540.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_with_memory = LLMChain(


=== With Memory (Updated Prompt) ===

Q1: My name is Alex.
A1: Nice to meet you, Alex! How can I assist you today?

Q2: What is my name?
A2: Your name is Alex.



## 5.3 — Inspecting Conversation History

Description:
In this cell, we’ll ask the LLM multiple questions and then print the stored chat history from ConversationBufferMemory.
This helps students understand what LangChain remembers and how it injects the history into the prompt.

In [19]:
# ======================================
# Section 5.3 — Inspecting Conversation History (Updated)
# ======================================

# Step 1: Ask multiple related questions
q1 = "I live in Sofia."
q2 = "My favorite programming language is Python."
q3 = "Where do I live and what is my favorite language?"

resp1 = chain_with_memory.invoke({"input": q1})
resp2 = chain_with_memory.invoke({"input": q2})
resp3 = chain_with_memory.invoke({"input": q3})

# Step 2: Show responses
print("=== Responses ===\n")
print(f"Q1: {q1}\nA1: {resp1['text']}\n")
print(f"Q2: {q2}\nA2: {resp2['text']}\n")
print(f"Q3: {q3}\nA3: {resp3['text']}\n")

# Step 3: Inspect stored chat history
print("=== Stored Conversation History ===\n")
for msg in memory.chat_memory.messages:
    role = "User" if msg.type == "human" else "Assistant"
    print(f"{role}: {msg.content}")


=== Responses ===

Q1: I live in Sofia.
A1: That's great, Alex! Sofia is a beautiful city with a rich history and culture. What do you enjoy most about living there?

Q2: My favorite programming language is Python.
A2: Python is a fantastic choice! It's known for its readability and versatility. What do you enjoy most about using Python?

Q3: Where do I live and what is my favorite language?
A3: You live in Sofia, and your favorite programming language is Python.

=== Stored Conversation History ===

User: My name is Alex.
Assistant: Nice to meet you, Alex! How can I assist you today?
User: What is my name?
Assistant: Your name is Alex.
User: I live in Sofia.
Assistant: That's great, Alex! Sofia is a beautiful city with a rich history and culture. What do you enjoy most about living there?
User: My favorite programming language is Python.
Assistant: Python is a fantastic choice! It's known for its readability and versatility. What do you enjoy most about using Python?
User: Where do I 

## 5.4 — Using ConversationBufferWindowMemory

Description:
By default, ConversationBufferMemory stores the entire chat history, which can become expensive for long conversations.
ConversationBufferWindowMemory solves this by keeping only the last N message pairs.

In [22]:
# ======================================
# Section 5.4 — Using ConversationBufferWindowMemory (Updated)
# ======================================

from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

# Step 1: Create a prompt that EXPLICITLY shows chat history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("system", "Only the remembered conversation is shown here:\n{chat_history}"),
    ("human", "{input}")
])

# Step 2: Create a windowed memory object (keep last 2 message pairs)
window_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=2,                    # Keep only last 2 message pairs
    return_messages=True    # Store structured messages instead of plain text
)

# Step 3: Create an LLM chain with windowed memory
chain_with_window_memory = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=window_memory
)

# Step 4: Ask multiple questions
q1 = "My name is Alex."
q2 = "I live in Sofia."
q3 = "I work at Ethera Technologies."
q4 = "What is my name?"

resp1 = chain_with_window_memory.invoke({"input": q1})
resp2 = chain_with_window_memory.invoke({"input": q2})
resp3 = chain_with_window_memory.invoke({"input": q3})
resp4 = chain_with_window_memory.invoke({"input": q4})

# Step 5: Print assistant responses
print("=== Responses ===\n")
print(f"Q1: {q1}\nA1: {resp1['text']}\n")
print(f"Q2: {q2}\nA2: {resp2['text']}\n")
print(f"Q3: {q3}\nA3: {resp3['text']}\n")
print(f"Q4: {q4}\nA4: {resp4['text']}\n")

# Step 6: Inspect what is ACTUALLY stored in memory
print("=== RAW MEMORY STATE (Last 2 Message Pairs) ===\n")
for msg in window_memory.chat_memory.messages:
    role = "User" if msg.type == "human" else "Assistant"
    print(f"{role}: {msg.content}")

# Step 7: Show the chat history variable injected into the prompt
print("\n=== CHAT HISTORY IN PROMPT ===\n")
chat_history_messages = window_memory.load_memory_variables({})["chat_history"]
for msg in chat_history_messages:
    role = "User" if msg.type == "human" else "Assistant"
    print(f"{role}: {msg.content}")

=== Responses ===

Q1: My name is Alex.
A1: Nice to meet you, Alex! How can I assist you today?

Q2: I live in Sofia.
A2: That's great, Alex! Sofia is a beautiful city with a rich history. What do you enjoy most about living there?

Q3: I work at Ethera Technologies.
A3: That sounds interesting, Alex! What do you do at Ethera Technologies?

Q4: What is my name?
A4: Your name is Alex.

=== RAW MEMORY STATE (Last 2 Message Pairs) ===

User: My name is Alex.
Assistant: Nice to meet you, Alex! How can I assist you today?
User: I live in Sofia.
Assistant: That's great, Alex! Sofia is a beautiful city with a rich history. What do you enjoy most about living there?
User: I work at Ethera Technologies.
Assistant: That sounds interesting, Alex! What do you do at Ethera Technologies?
User: What is my name?
Assistant: Your name is Alex.

=== CHAT HISTORY IN PROMPT ===

User: I work at Ethera Technologies.
Assistant: That sounds interesting, Alex! What do you do at Ethera Technologies?
User: What 

# Section 6 — Advanced Memory Types

In this section, we’ll demonstrate how LangChain offers multiple memory strategies depending on:

Chat length

Token cost

Desired behavior (remember everything vs. summarize vs. extract facts)

## Cell 6.1 — Using ConversationSummaryMemory

Goal:
Show students how LangChain can summarize previous conversations automatically
instead of storing the entire history.

This is useful when:

Conversations are very long.

You want the model to have compressed context.

In [25]:
# ======================================
# Section 6.1 — Using ConversationSummaryMemory
# ======================================

from langchain.memory import ConversationSummaryMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

# Step 1: Create the prompt with explicit {chat_history}
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("system", "Here is the summary of the conversation so far:\n{chat_history}"),
    ("human", "{input}")
])


summary_model = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_completion_tokens=50)

# Step 2: Create ConversationSummaryMemory
# ------------------------------------------------------
# Instead of storing full messages, it generates a summary using the LLM.
summary_memory = ConversationSummaryMemory(
    llm=llm,                   # Uses the same LLM for summarization
    memory_key="chat_history", # Injected into the prompt
    return_messages=True
)

# Step 3: Create a chain using summary memory
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=summary_memory
)

# Step 4: Ask multiple questions to demonstrate automatic summarization
q1 = "My name is Alex, and I live in Sofia."
q2 = "I work at Ethera Technologies as an AI consultant."
q3 = "Can you summarize my personal details?"

resp1 = summary_chain.invoke({"input": q1})
resp2 = summary_chain.invoke({"input": q2})
resp3 = summary_chain.invoke({"input": q3})

# Step 5: Print responses
print("=== Responses ===\n")
print(f"Q1: {q1}\nA1: {resp1['text']}\n")
print(f"Q2: {q2}\nA2: {resp2['text']}\n")
print(f"Q3: {q3}\nA3: {resp3['text']}\n")

# Step 6: Inspect the stored memory summary
print("\n=== STORED MEMORY (Summary) ===\n")
print(summary_memory.load_memory_variables({})["chat_history"])


=== Responses ===

Q1: My name is Alex, and I live in Sofia.
A1: Nice to meet you, Alex! How can I assist you today?

Q2: I work at Ethera Technologies as an AI consultant.
A2: That's great to hear, Alex! Working as an AI consultant must be quite interesting, especially with the rapid advancements in the field. What kind of projects or areas do you focus on at Ethera Technologies?

Q3: Can you summarize my personal details?
A3: Sure! Here are the personal details you've shared so far:

- Your name is Alex.
- You live in Sofia.
- You work at Ethera Technologies as an AI consultant.

If there's anything else you'd like to add or modify, just let me know!


=== STORED MEMORY (Summary) ===

[SystemMessage(content="The human introduces themselves as Alex and mentions they live in Sofia. The AI expresses pleasure in meeting Alex and asks how it can assist them today. Alex shares that they work at Ethera Technologies as an AI consultant. The AI then summarizes Alex's personal", additional_kwa

## 6.2 — Using ConversationKGMemory (Entity & Fact Memory)

Description:
ConversationKGMemory extracts structured facts (subject–predicate–object triplets) from the dialog (e.g., “Alex — lives_in — Sofia”).
This is useful when you want the assistant to remember stable facts about a user or topic without carrying the entire chat text.

In [26]:
# ======================================
# Section 6.2 — Using ConversationKGMemory (Fixed)
# ======================================

# 1) Use the community package (recommended in recent LangChain versions)
from langchain_community.memory.kg import ConversationKGMemory
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph  # for direct triples insight

from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

# 2) Prompt uses the *same* memory_key we give to KG memory below ("history")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("system", "Known facts extracted so far:\n{history}"),
    ("human", "{input}")
])

# 3) KG memory: align keys with LLMChain (input='input', output='text'); keep a few turns in view (k)
kg_memory = ConversationKGMemory(
    llm=llm,
    memory_key="history",   # <- matches prompt
    input_key="input",      # <- LLMChain input dict will have {"input": "..."}
    output_key="text",      # <- LLMChain default output key is "text"
    return_messages=False,  # you can set True if you want message objects instead of string
    k=4                     # consider last 4 utterances (2 pairs) when extracting
)

kg_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=kg_memory,
    output_key="text"       # explicit to match memory.output_key
)

# 4) Mini-dialog populating facts
u1 = "My name is Alex."
u2 = "I live in Sofia, Bulgaria."
u3 = "I work at Ethera Technologies as an AI consultant."
u4 = "What do you know about me so far?"

r1 = kg_chain.invoke({"input": u1})
r2 = kg_chain.invoke({"input": u2})
r3 = kg_chain.invoke({"input": u3})
r4 = kg_chain.invoke({"input": u4})

print("=== Responses ===\n")
print(f"U1: {u1}\nA1: {r1['text']}\n")
print(f"U2: {u2}\nA2: {r2['text']}\n")
print(f"U3: {u3}\nA3: {r3['text']}\n")
print(f"U4: {u4}\nA4: {r4['text']}\n")

# 5) Inspect the raw triples directly from the graph (bypasses the entity filter)
print("=== RAW TRIPLES IN KG ===")
print(kg_memory.kg.get_triples())  # list of (subject, relation, object)
# (If this prints [], the extractor didn't parse your inputs; try rephrasing u1–u3 slightly.)



=== Responses ===

U1: My name is Alex.
A1: Nice to meet you, Alex! How can I assist you today?

U2: I live in Sofia, Bulgaria.
A2: That's great! Sofia is a beautiful city with a rich history and vibrant culture. If you have any questions about Sofia or need information about anything specific, feel free to ask!

U3: I work at Ethera Technologies as an AI consultant.
A3: That's great! As an AI consultant at Ethera Technologies, you likely work on various projects involving artificial intelligence, machine learning, and data analysis. If you have any specific questions or topics you'd like to discuss related to your work or AI in general, feel free to ask!

U4: What do you know about me so far?
A4: I don't have any information about you. I don't have access to personal data unless you share it with me during our conversation. How can I assist you today?

=== RAW TRIPLES IN KG ===
[('Nevada', 'state', 'is a'), ('Sofia', 'Bulgaria', 'is in'), ('Ethera Technologies', 'company', 'is a'), ('

# Section 7 — LangChain Agents
Goal
Teach how LangChain agents work, what tools are, and how to build a simple example where an agent decides what to do step by step.

## Section 7.1 — Basic Agent with Built-in Tools

Description:
In this section, we introduce LangChain agents and demonstrate how to create a simple agent that can reason, choose between multiple tools, and combine their outputs to answer complex questions.
Students will learn:

How to initialize an agent with an LLM.

How to register built-in tools like Wikipedia search and Calculator.

How the agent decides which tool to use based on the tool descriptions.

How the agent combines results from multiple tools to provide a final answer.

The code cell I shared above stays the same — we only add this description at the start of the section.

In [ ]:
!pip install -U wikipedia
!pip install -U langchain-experimental

In [27]:
# ======================================
# Section 7.1 — Using Custom Tools (Function Calling)
# ======================================

from langchain.tools import StructuredTool
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain.schema.runnable import RunnablePassthrough

# -------------------------
# Step 1: Custom Function
# -------------------------
def get_weather(city: str) -> str:
    """Simulated weather info provider."""
    fake_data = {
        "Sofia": "Sunny, 28°C",
        "Plovdiv": "Partly cloudy, 25°C",
        "Varna": "Windy, 22°C"
    }
    return fake_data.get(city, f"Weather data for {city} is not available.")

# -------------------------
# Step 2: Wrap the Function as a LangChain Tool
# -------------------------
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="WeatherTool",
    description="Use this tool to get the current weather for a specific city."
)

# -------------------------
# Step 3: Prompt to Extract City Name
# -------------------------
city_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract ONLY the city name from the user's message."),
    ("human", "{input}")
])

# -------------------------
# Step 4: Create the Routing Function
# -------------------------
def route_question(inputs: dict):
    """
    Decides whether to call the weather tool or the LLM.

    Parameters:
        inputs (dict):
            - "input" (str): The user question.

    Returns:
        dict: A dictionary with {"response": <str>} containing the assistant's reply.
    """
    query = inputs["input"].lower()

    # If the user asks about weather → extract the city → call tool
    if "weather" in query:
        city = llm.predict(city_prompt.format(input=inputs["input"])).strip()
        return {"response": weather_tool.func(city)}

    # Otherwise → just answer normally with LLM
    return {"response": llm.predict(inputs["input"])}

# -------------------------
# Step 5: Build the Chain
# -------------------------
chain = RunnableMap({
    "input": RunnablePassthrough(),
    "response": route_question
})

# -------------------------
# Step 6: Test It
# -------------------------
questions = [
    "What's the weather in Sofia?",
    "What's the weather in Varna?",
    "Who founded OpenAI?"
]

for q in questions:
    # ✅ Now we pass a dict to chain.invoke()
    result = chain.invoke({"input": q})
    print(f"\nUser: {q}\nAssistant: {result['response']}")


/tmp/ipython-input-1272579295.py:57: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  city = llm.predict(city_prompt.format(input=inputs["input"])).strip()



User: What's the weather in Sofia?
Assistant: {'response': 'Sunny, 28°C'}

User: What's the weather in Varna?
Assistant: {'response': 'Windy, 22°C'}

User: Who founded OpenAI?
Assistant: {'response': 'OpenAI was founded in December 2015 by Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, Wojciech Zaremba, and John Schulman, among others. The organization was established with the goal of advancing artificial intelligence in a way that is safe and beneficial for humanity.'}
